In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.transform import *
from local.data.core import *
from local.data.source import *
from local.data.external import *
from local.data.pipeline import *
from local.text.core import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp text.data
#default_cls_lvl 3

# Text data

> Functions and transforms to help gather text data in a `DataSource

## Numericalizing

In [ ]:
def make_vocab(count, min_freq=3, max_vocab=60000):
    "Create a vocab of `max_vocab` size from `Counter` `count` with items present more than `min_freq`"
    vocab = [o for o,c in count.most_common(max_vocab) if c >= min_freq]
    for o in reversed(defaults.text_spec_tok): #Make sure all special tokens are in the vocab
        if o in vocab: vocab.remove(o)
        vocab.insert(0, o)
    vocab = vocab[:max_vocab]
    if len(vocab) < max_vocab and len(vocab)%8 != 0: 
        #Make sure vocab size is a multiple of 8 for fast mixed precision training
        vocab += ['xxfake' for _ in range(0, 8-len(vocab)%8)]
    return vocab

In [ ]:
count = Counter(['a', 'a', 'a', 'a', 'b', 'b', 'c', 'c', 'd'])
test_eq(set(make_vocab(count)), set(defaults.text_spec_tok + 'a xxfake'.split()))
test_eq(len(make_vocab(count))%8, 0)
test_eq(set(make_vocab(count, min_freq=1)), set(defaults.text_spec_tok + 'a b c d xxfake'.split()))
test_eq(set(make_vocab(count,max_vocab=12, min_freq=1)), set(defaults.text_spec_tok + 'a b c'.split()))

In [ ]:
# export
class Numericalize(ItemTransform):
    "Reversible transform of tokenized texts to numericalized ids"
    def __init__(self, vocab=None, min_freq=3, max_vocab=60000, sep=None):
        self.sep = sep or defaults.text_token_sep
        self.vocab,self.min_freq,self.max_vocab = vocab,min_freq,max_vocab
        self.o2i = None if vocab is None else defaultdict(int, {v:k for k,v in enumerate(vocab)})
    
    def setup(self, dsrc):
        if dsrc is None: return
        if self.vocab is None:
            dsrc = getattr(dsrc,'train',dsrc)
            count = Counter(p for o in dsrc for p in o.split(self.sep))
            self.vocab = make_vocab(count, min_freq=self.min_freq, max_vocab=self.max_vocab)
            self.o2i = defaultdict(int, {v:k for k,v in enumerate(self.vocab) if v != 'xxfake'})

    def encodes(self, o):      return [self.o2i[o_] for o_ in o.split(self.sep)]
    def decodes(self, o)->Str: return self.sep.join([self.vocab[o_] for o_ in o])

In [ ]:
num = Numericalize(min_freq=1, sep=' ')
num.setup(L('This is an example of text', 'this is another text'))
test_eq(set(num.vocab), set(defaults.text_spec_tok + 'This is an example of text this another xxfake'.split()))
test_eq(len(num.vocab)%8, 0)
start = 'This is an example of text'
t = num(start)
test_eq(t, [11, 9, 12, 13, 14, 10])
test_eq(num.decode(t), start)

In [ ]:
num = Numericalize(min_freq=2, sep=' ')
num.setup(L('This is an example of text', 'this is another text'))
test_eq(set(num.vocab), set(defaults.text_spec_tok + 'is text xxfake'.split()))
test_eq(len(num.vocab)%8, 0)
t = num(start)
test_eq(t, [0, 9, 0, 0, 0, 10])
test_eq(num.decode(t), f'{UNK} is {UNK} {UNK} {UNK} text')

## LMPreloader -

In [ ]:
class LMSampler(Sampler):
    

In [ ]:
class LM_PreLoader(GetAttr):
    "An intermediate between a dataset with texts and a DataLoader"
    _xtra = ['show', 'decode', 'show_at', 'decode_at', 'decode_batch']
    def __init__(self, ds, lengths=None, bs=64, seq_len=70, shuffle=False):
        self.ds,self.bs,self.seq_len,self.shuffle = ds,bs,seq_len,shuffle
        self.lengths = [len(o[0]) for o in ds] if lengths is None else lengths
        self.n_batch = sum(self.lengths) // bs
        self.batchify()
        self.default = self.ds
    
    def __len__(self): return ((self.n_batch-1) // self.seq_len) * self.bs
    
    def __getitem__(self, i):
        k = (i % self.bs) * self.n_batch + (i // self.bs) * self.seq_len
        item_idx = (self.cumlen > k).nonzero().min().item()
        offset = k if item_idx==0 else k-self.cumlen[item_idx-1]
        text = self.ds[item_idx][0][offset:]
        while len(text) <= self.seq_len:
            item_idx += 1
            text += self.ds[item_idx][0]
        return tensor(text[:self.seq_len]),tensor(text[1:self.seq_len+1])
    
    def batchify(self):
        self.idxs = torch.randperm(len(ds)) if self.shuffle else tensor(range(len(self.ds)))
        self.cumlen = (tensor(self.lengths)[idxs] if self.shuffle else tensor(self.lengths)).cumsum(0)

In [ ]:
lengths = [10,7,19,23,5,42]
ds = LM_PreLoader([(list(range(l)), 0) for l in lengths], lengths=lengths, bs=5, seq_len=4)
x,y = ds[0]
test_eq(x[1:], y[:-1])
test_eq(x+1, y)
#Going on the seq dimension reads the text in order
test_eq(torch.cat([ds[5*i][0] for i in range(5)]), 
        tensor(list(range(10))+list(range(7))+list(range(3))))
#3 is skipped for the next sample in the natch since it's the last target
test_eq(torch.cat([ds[5*i+1][0] for i in range(5)]),
        tensor(list(range(4,19))+list(range(5))))

## Integration example

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')

In [ ]:
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [ ]:
df_tok,count = tokenize_df(df, 'text')

In [ ]:
df_tok.head()

,label,is_valid,text,text_lengths
0,negative,False,xxbos▁xxmaj▁un▁-▁bleeping▁-▁believable▁!▁xxmaj...,103.0
1,positive,False,xxbos▁xxmaj▁this▁is▁a▁extremely▁well▁-▁made▁fi...,462.0
2,negative,False,xxbos▁xxmaj▁every▁once▁in▁a▁long▁while▁a▁movie...,220.0
3,positive,False,xxbos▁xxmaj▁name▁just▁says▁it▁all▁.▁i▁watched▁...,184.0
4,negative,False,xxbos▁xxmaj▁this▁movie▁succeeds▁at▁being▁one▁o...,398.0


In [ ]:
texts,lengths = df_tok['text'].values,df_tok['text_lengths'].map(int).values

In [ ]:
splits = RandomSplitter()(L(t for t in texts))
dsrc = DataSource(L(t for t in texts), type_tfms=[Numericalize(make_vocab(count))], filts=splits)

In [ ]:
dsrc.decode_at(0)

("xxbos▁xxmaj▁un▁-▁xxunk▁-▁believable▁!▁xxmaj▁meg▁xxmaj▁ryan▁does▁n't▁even▁look▁her▁usual▁xxunk▁lovable▁self▁in▁this▁,▁which▁normally▁makes▁me▁forgive▁her▁shallow▁xxunk▁acting▁xxunk▁.▁xxmaj▁hard▁to▁believe▁she▁was▁the▁producer▁on▁this▁dog▁.▁xxmaj▁plus▁xxmaj▁kevin▁xxmaj▁kline▁:▁what▁kind▁of▁suicide▁trip▁has▁his▁career▁been▁on▁?▁xxmaj▁xxunk▁...▁xxmaj▁xxunk▁!▁!▁!▁xxmaj▁finally▁this▁was▁directed▁by▁the▁guy▁who▁did▁xxmaj▁big▁xxmaj▁xxunk▁?▁xxmaj▁must▁be▁a▁replay▁of▁xxmaj▁jonestown▁-▁hollywood▁style▁.▁xxmaj▁xxunk▁!",)

In [ ]:
len(splits[0])

800

In [ ]:
bs = 16
ds = LM_PreLoader(dsrc.train, lengths=lengths[splits[0]], bs=bs)
dl = TfmdDL(ds, bs=bs, num_workers=0)

In [ ]:
x,y = dl.one_batch()

In [ ]:
dl.decode_batch((x,y))

(#10) [('xxbos▁xxmaj▁rosie▁wasted▁a▁lot▁of▁xxup▁tv▁time▁talking▁about▁the▁xxmaj▁xxunk▁as▁if▁they▁were▁super▁xxunk▁in▁the▁xxunk▁of▁the▁modern▁day▁xxmaj▁puerto▁xxmaj▁xxunk▁.▁xxmaj▁they▁were▁not▁.▁xxmaj▁the▁truth▁is▁that▁the▁xxmaj▁africans▁and▁the▁xxmaj▁spanish▁were▁and▁she▁knows▁it▁.▁xxmaj▁what▁kills▁me▁is▁that▁she▁is▁standing▁on▁the▁screen▁looking▁like',),('it▁and▁"▁office▁xxmaj▁space▁"▁.▁xxmaj▁blockbuster▁and▁xxup▁imdb▁even▁had▁it▁as▁an▁"▁also▁recommended▁"▁selection▁if▁you▁liked▁"▁office▁xxmaj▁space▁"▁.▁\n\n▁xxmaj▁now▁,▁xxmaj▁i▁\'ve▁seen▁xxmaj▁office▁xxmaj▁space▁probably▁15▁or▁20▁times▁.▁i▁love▁it▁.▁xxmaj▁it▁\'s▁probably▁one▁of▁my▁10▁favorite▁movies▁.▁xxmaj▁witty▁,▁humorous▁,',),('be▁the▁xxmaj▁ed▁xxmaj▁wood▁of▁xxmaj▁asia▁,▁made▁two▁unfortunate▁decisions▁involving▁sound▁.▁xxmaj▁first▁,▁he▁choose▁to▁use▁a▁soundtrack▁only▁when▁someone▁is▁about▁to▁be▁killed▁.▁xxmaj▁this▁is▁an▁excellent▁xxunk▁for▁xxunk▁any▁suspense▁because▁the▁audience▁gets▁a▁two▁minute▁warning▁to▁xxunk▁for▁another▁miserab

In [ ]:
ds.decode((x[0],))

('xxbos▁xxmaj▁rosie▁wasted▁a▁lot▁of▁xxup▁tv▁time▁talking▁about▁the▁xxmaj▁xxunk▁as▁if▁they▁were▁super▁xxunk▁in▁the▁xxunk▁of▁the▁modern▁day▁xxmaj▁puerto▁xxmaj▁xxunk▁.▁xxmaj▁they▁were▁not▁.▁xxmaj▁the▁truth▁is▁that▁the▁xxmaj▁africans▁and▁the▁xxmaj▁spanish▁were▁and▁she▁knows▁it▁.▁xxmaj▁what▁kills▁me▁is▁that▁she▁is▁standing▁on▁the▁screen▁looking▁like',)